In [1]:
import pathlib
import itertools
import collections

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model, model_selection
from sklearn import cross_decomposition
from sklearn import decomposition
from sklearn import pipeline

In [2]:
data_folder = pathlib.Path("./../data/spam")

In [3]:
list(data_folder.iterdir())

[PosixPath('../data/spam/spam.info.txt'),
 PosixPath('../data/spam/spam.data'),
 PosixPath('../data/spam/spam.traintest')]

In [4]:
info_file = data_folder / "spam.info.txt"
data_file = data_folder / "spam.data"

### Data info

In [5]:
print(info_file.read_text())

1. Title:  SPAM E-mail Database

2. Sources:
   (a) Creators: Mark Hopkins, Erik Reeber, George Forman, Jaap Suermondt
        Hewlett-Packard Labs, 1501 Page Mill Rd., Palo Alto, CA 94304
   (b) Donor: George Forman (gforman at nospam hpl.hp.com)  650-857-7835
   (c) Generated: June-July 1999

3. Past Usage:
   (a) Hewlett-Packard Internal-only Technical Report. External forthcoming.
   (b) Determine whether a given email is spam or not.
   (c) ~7% misclassification error.
       False positives (marking good mail as spam) are very undesirable.
       If we insist on zero false positives in the training/testing set,
       20-25% of the spam passed through the filter.

4. Relevant Information:
        The "spam" concept is diverse: advertisements for products/web
        sites, make money fast schemes, chain letters, pornography...
	Our collection of spam e-mails came from our postmaster and 
	individuals who had filed spam.  Our collection of non-spam 
	e-mails came from filed work a

In [6]:
df = pd.read_csv(data_file, header=None, sep=" ")
df.rename({57: "label"}, axis=1, inplace=True)
df.head()

0     1     2    3     4     5     6     7     8     9  ...    48  \
0  0.00  0.64  0.64  0.0  0.32  0.00  0.00  0.00  0.00  0.00  ...  0.00   
1  0.21  0.28  0.50  0.0  0.14  0.28  0.21  0.07  0.00  0.94  ...  0.00   
2  0.06  0.00  0.71  0.0  1.23  0.19  0.19  0.12  0.64  0.25  ...  0.01   
3  0.00  0.00  0.00  0.0  0.63  0.00  0.31  0.63  0.31  0.63  ...  0.00   
4  0.00  0.00  0.00  0.0  0.63  0.00  0.31  0.63  0.31  0.63  ...  0.00   

      49   50     51     52     53     54   55    56  label  
0  0.000  0.0  0.778  0.000  0.000  3.756   61   278      1  
1  0.132  0.0  0.372  0.180  0.048  5.114  101  1028      1  
2  0.143  0.0  0.276  0.184  0.010  9.821  485  2259      1  
3  0.137  0.0  0.137  0.000  0.000  3.537   40   191      1  
4  0.135  0.0  0.135  0.000  0.000  3.537   40   191      1  

[5 rows x 58 columns]

In [7]:
train_df, test_df = model_selection.train_test_split(df, train_size=0.3)
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [8]:
def cal_z_score(df, y_hat, train_cols, label_col, coefficients):
    y = df.loc[:, label_col].values
    X = df.loc[:, train_cols].values
    num_ele = y.shape[0]
    num_preds = X.shape[1]
    
    S = np.linalg.inv(np.matmul(X.T, X))
    var = np.square(y-y_hat).sum()/(num_ele - num_preds - 1)
    
    z_scores = []
    for i, coeff in enumerate(coefficients):
        z_score = coeff/(S[i][i]*np.sqrt(S[i][i]))
        z_scores.append(z_score)
        
    return z_scores

cal_rss = lambda y, y_hat: np.square(y - y_hat).sum()/y.shape[0]

In [9]:
def train_OLS(train_df, preds_cols, label_col):
    reg = linear_model.LinearRegression(n_jobs=-1)
    reg.fit(train_df.loc[:, preds_cols], train_df.loc[:, label_col])
    return reg, preds_cols, label_col

In [10]:
reg_dict = {}

In [11]:
target = "label"
predictors = list(train_df.columns)
predictors.remove("label")

# OLS

In [12]:
ols_set = train_OLS(train_df, predictors, target)
reg_dict["ols"] = {"reg": ols_set[0], "cols": ols_set[1], "coeff": ols_set[0].coef_}

# Best subset selector

In [13]:
best_set = None
best_set_rss = -1
best_n = 3
for i in range(1, best_n):
    for cols in itertools.combinations(predictors, i):
        fit_set = train_OLS(train_df, cols, target)
        
        ## CALCULATE METRICS ON TEST SET
        y = test_df.loc[:, fit_set[2]]
        X = test_df.loc[:, fit_set[1]]
        
        y_hat = fit_set[0].predict(X)
        fit_rss = cal_rss(y, y_hat)
        if fit_rss > best_set_rss:
            best_set = fit_set
            best_set_rss = fit_rss

In [14]:
## Cal best subset scores
X = test_df.loc[:, best_set[1]]
y = test_df.loc[:, best_set[2]]

reg_dict["best subset"] = {"reg": best_set[0], "cols": best_set[1], "coeff": best_set[0].coef_}

# Ridge

In [15]:
ridge = linear_model.Ridge()
ridge.fit(train_df.loc[:, predictors], train_df.loc[:, target])

Ridge()

In [16]:
reg_dict["ridge"] = {"reg": ridge, "cols": predictors, "coeff": ridge.coef_}

# Lasso

In [17]:
lasso = linear_model.Lasso()
lasso.fit(train_df.loc[:, predictors], train_df.loc[:, target])

Lasso()

In [18]:
reg_dict["lasso"] = {"reg": lasso, "cols": predictors, "coeff": lasso.coef_}

# PCR

In [19]:
n_comps = len(predictors)
pcr = pipeline.Pipeline([
    ('pca', decomposition.PCA(n_components=n_comps)), 
    ('ols', linear_model.LinearRegression()
    )]
        )
pcr.fit(train_df.loc[:, predictors], train_df.loc[:, target])

Pipeline(steps=[('pca', PCA(n_components=57)), ('ols', LinearRegression())])

In [20]:
reg_dict["pcr"] = {"reg": pcr, "cols": predictors, "coeff": pcr.named_steps["ols"].coef_}

# PLS

In [21]:
n_comps = len(predictors)
pls = cross_decomposition.PLSRegression(n_components=n_comps)
pls.fit(train_df.loc[:, predictors], train_df.loc[:, target])

PLSRegression(n_components=57)

In [36]:
reg_dict["pls"] = {"reg": pls, "cols": predictors, "coeff": [coef[0] for coef in pls.coef_]}

# Collate results

In [42]:
results_dict = {key: {
                        value["cols"][idx]: value["coeff"][idx] 
                        for idx in range(len(value["cols"]))
                     }
                for key, value in reg_dict.items()}

results_dict = {key: {**value, **{"avg" : sum(value.values())}}
                for key, value in results_dict.items()}

In [43]:
results = pd.DataFrame.from_dict(results_dict)

In [44]:
results

ols  best subset     ridge     lasso       pcr       pls
0   -0.043206          NaN -0.043765  0.000000  0.000210 -0.012157
1   -0.012887          NaN -0.012716 -0.000000  0.000843 -0.017152
2    0.014509          NaN  0.015208  0.000000 -0.002523  0.006994
3    0.009416          NaN  0.009455  0.000000 -0.025712  0.009189
4    0.090294          NaN  0.089911  0.000000 -0.084523  0.056738
5    0.128448          NaN  0.127940  0.000000  0.029253  0.033803
6    0.213599          NaN  0.213686  0.000000 -0.010038  0.081964
7    0.084915          NaN  0.084343  0.000000  0.103894  0.038610
8    0.079565          NaN  0.080278  0.000000 -0.051765  0.020950
9    0.032111          NaN  0.032245  0.000000  0.028895  0.019806
10  -0.008772          NaN -0.010315  0.000000  0.075756 -0.001960
11  -0.023717          NaN -0.023765  0.000000  0.038491 -0.019223
12   0.004958          NaN  0.007217  0.000000  0.070760  0.001673
13   0.014639          NaN  0.015690  0.000000 -0.008725  0.004304
14   0.069478          NaN  0.070116  0.000000 -0.041245  0.016503
15   0.061446          NaN  0.061057  0.000000  0.024483  0.057430
16   0.034106          NaN  0.033768  0.000000 -0.035986  0.014878
17   0.037441          NaN  0.037369  0.000000 -0.100024  0.019453
18   0.020024          NaN  0.019601  0.000000  0.040704  0.035309
19   0.057542          NaN  0.057988  0.000000  0.114007  0.038148
20   0.057981          NaN  0.058762  0.000000 -0.087815  0.072366
21   0.046359          NaN  0.046067  0.000000 -0.107393  0.042529
22   0.143944          NaN  0.146266  0.000000  0.031807  0.043207
23   0.064749          NaN  0.065100  0.000000  0.123851  0.033235
24  -0.020043          NaN -0.020129 -0.000000  0.030364 -0.036559
25  -0.021037          NaN -0.021244 -0.000000  0.132523 -0.019044
26  -0.011812          NaN -0.011849 -0.000000 -0.029531 -0.039864
27   0.018381          NaN  0.017696 -0.000000 -0.031546  0.010110
28  -0.055988          NaN -0.055408 -0.000000 -0.040350 -0.023126
29  -0.082282          NaN -0.081075 -0.000000 -0.034418 -0.038309
30  -0.066188          NaN -0.059352 -0.000000 -0.068540 -0.022257
31   1.087473          NaN  0.232659 -0.000000  0.061291  0.348321
32  -0.035980          NaN -0.035966 -0.000000  0.227059 -0.025162
33  -0.973703          NaN -0.120670 -0.000000  0.013854 -0.311822
34  -0.092340          NaN -0.092769 -0.000000 -0.103510 -0.040604
35   0.089599          NaN  0.086919 -0.000000 -0.058537  0.035306
36  -0.020540          NaN -0.020527 -0.000000 -0.025143 -0.008129
37  -0.083151          NaN -0.081472 -0.000000 -0.157492 -0.019202
38  -0.047560          NaN -0.047582 -0.000000 -0.042038 -0.017691
39   0.077903          NaN  0.074580 -0.000000  0.078467  0.025457
40  -0.028147          NaN -0.028104 -0.000000  0.107296 -0.008981
41  -0.032964          NaN -0.033267 -0.000000  0.036972 -0.023297
42  -0.062932          NaN -0.062811 -0.000000 -0.002603 -0.013312
43  -0.039360          NaN -0.039318 -0.000000 -0.136974 -0.021776
44  -0.059044          NaN -0.058976 -0.000000  0.081757 -0.051655
45  -0.034424          NaN -0.034452 -0.000000  0.014953 -0.034347
46  -0.127176          NaN -0.110237 -0.000000  0.003083 -0.008753
47  -0.067046          NaN -0.065621 -0.000000 -0.149271 -0.015332
48  -0.145241          NaN -0.143035 -0.000000 -0.027068 -0.039461
49   0.000558          NaN  0.000821 -0.000000 -0.068642  0.000133
50  -0.018010          NaN -0.018176 -0.000000  0.011843 -0.002735
51   0.160335          NaN  0.160025  0.000000 -0.070110  0.089176
52   0.235710          NaN  0.225424  0.000000 -0.008186  0.061441
53   0.057132          NaN  0.055988  0.000000 -0.084816  0.020205
54   0.000133    -0.002715  0.000146 -0.000170 -0.021589  0.004289
55   0.000050     0.001630  0.000046  0.000856 -0.127355  0.005973
56   0.000081          NaN  0.000080  0.000120  1.458241  0.045094
avg  0.779329    -0.001085  0.793849  0.000806  1.097186  0.420685